# Lab 3.3.2 Solutions: SGLang Deployment with RadixAttention

Complete solutions to all exercises from the SGLang deployment lab.

## Setup

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path("../scripts").resolve()))

import requests
import time
import json
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from benchmark_utils import InferenceBenchmark, BenchmarkConfig

---

## Exercise 1: Design Prefix Cache Patterns

**Task**: Design a prompt template system that maximizes prefix cache hits for a customer service chatbot.

In [ ]:
class PrefixOptimizedPromptSystem:
    """
    A prompt template system designed to maximize RadixAttention cache hits.
    
    Design Principles:
    1. Static content at the beginning (always cached)
    2. Semi-static content next (often cached)
    3. Dynamic content at the end (never cached)
    """
    
    # Level 1: Always the same (100% cache hit after first request)
    SYSTEM_PREFIX = """You are a helpful customer service assistant for TechCorp Inc.
Our products include:
- TechWidget Pro: Advanced productivity tool, $299, 2-year warranty
- TechWidget Lite: Entry-level option, $99, 1-year warranty  
- TechCloud: Cloud storage service, $9.99/month, 1TB storage
- TechSupport: Premium support package, $49.99/month

Company Policies:
- 30-day money-back guarantee on all hardware
- Free shipping on orders over $50
- Support hours: 24/7 for Premium, 9AM-6PM EST for Standard

Always be polite, helpful, and accurate. If unsure, offer to escalate.

"""
    
    # Level 2: Category-specific prefixes (high cache hit within category)
    CATEGORY_PREFIXES = {
        "billing": """Current context: BILLING INQUIRY
Relevant policies:
- Refunds processed within 5-7 business days
- Credit card issues should be escalated to finance
- Payment plans available for orders over $200

""",
        "technical": """Current context: TECHNICAL SUPPORT
Troubleshooting steps:
1. Always ask for product model and firmware version
2. Check if warranty is still valid
3. Try basic reset procedures first
4. Escalate hardware issues to Level 2 support

""",
        "shipping": """Current context: SHIPPING INQUIRY
Shipping information:
- Standard: 5-7 business days
- Express: 2-3 business days (+$15)
- Overnight: Next business day (+$35)
- International: 10-15 business days

""",
        "returns": """Current context: RETURNS & EXCHANGES
Return policies:
- 30-day return window for unopened items
- 14-day return window for opened items (restocking fee may apply)
- Defective items: Full refund or exchange anytime under warranty
- Return shipping is free for defective items

"""
    }
    
    def __init__(self):
        self.cache_stats = {
            "l1_hits": 0,  # System prefix reuse
            "l2_hits": 0,  # Category prefix reuse
            "total_requests": 0
        }
    
    def build_prompt(self, category: str, customer_message: str, 
                     customer_info: dict = None) -> str:
        """
        Build a prompt optimized for prefix caching.
        
        Structure:
        [STATIC: System Prefix] → Always cached after first request
        [SEMI-STATIC: Category Prefix] → Cached within category
        [DYNAMIC: Customer Context + Message] → Never cached
        """
        # Start with static system prefix
        prompt = self.SYSTEM_PREFIX
        
        # Add category-specific prefix
        if category in self.CATEGORY_PREFIXES:
            prompt += self.CATEGORY_PREFIXES[category]
        
        # Add dynamic customer context (if provided)
        if customer_info:
            prompt += f"Customer ID: {customer_info.get('id', 'Unknown')}\n"
            prompt += f"Account Type: {customer_info.get('tier', 'Standard')}\n"
            if customer_info.get('previous_issues'):
                prompt += f"Previous Issues: {customer_info['previous_issues']}\n"
            prompt += "\n"
        
        # Add the customer message (always last, always dynamic)
        prompt += f"Customer Message: {customer_message}\n\n"
        prompt += "Assistant Response:"
        
        self.cache_stats["total_requests"] += 1
        return prompt
    
    def estimate_cache_efficiency(self, prompts: list) -> dict:
        """
        Estimate cache efficiency for a batch of prompts.
        """
        # Count shared prefix lengths
        system_prefix_len = len(self.SYSTEM_PREFIX)
        total_chars = sum(len(p) for p in prompts)
        
        # All prompts share system prefix
        shared_chars = system_prefix_len * len(prompts)
        
        # Group by category to estimate category prefix sharing
        # (In real usage, you'd track this from build_prompt calls)
        
        return {
            "total_chars": total_chars,
            "shared_system_chars": shared_chars,
            "estimated_cache_ratio": shared_chars / total_chars if total_chars > 0 else 0
        }

# Demonstrate the system
prompt_system = PrefixOptimizedPromptSystem()

# Generate sample prompts
test_cases = [
    ("billing", "Why was I charged twice?"),
    ("billing", "Can I get a refund?"),
    ("technical", "My TechWidget Pro won't turn on"),
    ("technical", "How do I update the firmware?"),
    ("shipping", "Where is my order?"),
    ("returns", "I want to return my product"),
]

prompts = []
for category, message in test_cases:
    prompt = prompt_system.build_prompt(category, message)
    prompts.append(prompt)

# Analyze cache efficiency
efficiency = prompt_system.estimate_cache_efficiency(prompts)

print("📊 Prefix Cache Efficiency Analysis")
print("=" * 50)
print(f"Total prompts: {len(prompts)}")
print(f"Total characters: {efficiency['total_chars']:,}")
print(f"Shared system prefix chars: {efficiency['shared_system_chars']:,}")
print(f"Estimated cache ratio: {efficiency['estimated_cache_ratio']:.1%}")
print("\n💡 With RadixAttention, the system prefix is computed once")
print("   and reused for ALL subsequent requests!")

---

## Exercise 2: Benchmark Prefix Cache Hit Rates

**Task**: Write code to measure and visualize prefix cache hit rates across different prompt patterns.

In [ ]:
import time
from dataclasses import dataclass
from typing import List, Tuple

@dataclass
class CacheBenchmarkResult:
    """Results from a cache benchmark run."""
    pattern_name: str
    first_request_ms: float
    avg_subsequent_ms: float
    speedup_ratio: float
    estimated_cache_hit_rate: float

class PrefixCacheBenchmark:
    """
    Benchmark prefix cache effectiveness by measuring latency patterns.
    
    Cache hits show as: lower TTFT on subsequent requests with same prefix.
    """
    
    def __init__(self, base_url: str = "http://localhost:30000"):
        self.base_url = base_url
        self.session = requests.Session()
    
    def _measure_ttft(self, prompt: str, max_tokens: int = 10) -> float:
        """Measure time-to-first-token for a prompt."""
        start = time.perf_counter()
        
        response = self.session.post(
            f"{self.base_url}/v1/completions",
            json={
                "model": "default",
                "prompt": prompt,
                "max_tokens": max_tokens,
                "stream": True
            },
            stream=True
        )
        
        # TTFT is time to first chunk
        for line in response.iter_lines():
            if line:
                ttft = time.perf_counter() - start
                # Consume rest of response
                for _ in response.iter_lines():
                    pass
                return ttft * 1000  # Convert to ms
        
        return (time.perf_counter() - start) * 1000
    
    def benchmark_pattern(self, pattern_name: str, 
                          prompts: List[str],
                          runs_per_prompt: int = 3) -> CacheBenchmarkResult:
        """
        Benchmark a pattern of prompts for cache effectiveness.
        
        Args:
            pattern_name: Description of the pattern
            prompts: List of prompts (should share prefixes for cache hits)
            runs_per_prompt: Number of times to run each prompt
        """
        all_first_runs = []
        all_subsequent_runs = []
        
        for prompt in prompts:
            for run in range(runs_per_prompt):
                ttft = self._measure_ttft(prompt)
                
                if run == 0:
                    all_first_runs.append(ttft)
                else:
                    all_subsequent_runs.append(ttft)
        
        avg_first = np.mean(all_first_runs)
        avg_subsequent = np.mean(all_subsequent_runs) if all_subsequent_runs else avg_first
        
        # Speedup indicates cache effectiveness
        speedup = avg_first / avg_subsequent if avg_subsequent > 0 else 1.0
        
        # Estimate cache hit rate from speedup
        # If speedup > 1.5, likely getting cache hits
        estimated_hit_rate = min(1.0, max(0, (speedup - 1.0) / 0.5))
        
        return CacheBenchmarkResult(
            pattern_name=pattern_name,
            first_request_ms=avg_first,
            avg_subsequent_ms=avg_subsequent,
            speedup_ratio=speedup,
            estimated_cache_hit_rate=estimated_hit_rate
        )
    
    def run_full_benchmark(self) -> List[CacheBenchmarkResult]:
        """Run benchmark across different prefix sharing patterns."""
        results = []
        
        # Pattern 1: Identical prefix (100% cache potential)
        shared_prefix = "You are a helpful AI assistant. " * 50
        pattern1_prompts = [
            shared_prefix + "What is Python?",
            shared_prefix + "What is JavaScript?",
            shared_prefix + "What is Rust?",
        ]
        results.append(self.benchmark_pattern(
            "Identical Prefix (100% shared)", 
            pattern1_prompts
        ))
        
        # Pattern 2: Partial prefix sharing (50% cache potential)
        half_prefix = "You are a helpful AI assistant. " * 25
        pattern2_prompts = [
            half_prefix + "Context A: " + "x" * 100 + " Question: What is AI?",
            half_prefix + "Context B: " + "y" * 100 + " Question: What is ML?",
            half_prefix + "Context C: " + "z" * 100 + " Question: What is DL?",
        ]
        results.append(self.benchmark_pattern(
            "Partial Prefix (50% shared)", 
            pattern2_prompts
        ))
        
        # Pattern 3: No shared prefix (0% cache potential)
        pattern3_prompts = [
            "Context Alpha: " + "a" * 200 + " What is Python?",
            "Context Beta: " + "b" * 200 + " What is Java?",
            "Context Gamma: " + "c" * 200 + " What is C++?",
        ]
        results.append(self.benchmark_pattern(
            "No Shared Prefix (0% shared)", 
            pattern3_prompts
        ))
        
        return results

def visualize_cache_results(results: List[CacheBenchmarkResult]):
    """Create ASCII visualization of cache benchmark results."""
    print("\n📊 Prefix Cache Benchmark Results")
    print("=" * 70)
    print(f"{'Pattern':<30} {'1st Req':>10} {'Cached':>10} {'Speedup':>10} {'Hit Rate':>10}")
    print("-" * 70)
    
    for r in results:
        hit_bar = "█" * int(r.estimated_cache_hit_rate * 10)
        print(f"{r.pattern_name:<30} {r.first_request_ms:>8.1f}ms {r.avg_subsequent_ms:>8.1f}ms "
              f"{r.speedup_ratio:>9.2f}x {hit_bar}")
    
    print("\n💡 Key Insights:")
    best = max(results, key=lambda x: x.speedup_ratio)
    print(f"   • Best pattern: {best.pattern_name} ({best.speedup_ratio:.2f}x speedup)")
    print(f"   • Design prompts with shared prefixes to maximize cache hits")
    print(f"   • Put dynamic content at the END of prompts")

# Example usage (requires SGLang server running)
print("🔧 Prefix Cache Benchmark")
print("   Run this after starting SGLang server:")
print("   python -m sglang.launch_server --model Qwen/Qwen3-8B-Instruct")
print("")
print("   Example output with simulated data:")

# Simulated results for demonstration
simulated_results = [
    CacheBenchmarkResult("Identical Prefix (100%)", 120.5, 45.2, 2.67, 1.0),
    CacheBenchmarkResult("Partial Prefix (50%)", 115.3, 72.1, 1.60, 0.6),
    CacheBenchmarkResult("No Shared Prefix (0%)", 118.7, 115.2, 1.03, 0.0),
]
visualize_cache_results(simulated_results)

---

## Exercise 3: Multi-Turn Optimization

**Task**: Implement optimized multi-turn conversation handling that preserves RadixAttention benefits.

In [ ]:
from typing import List, Dict, Optional
import hashlib

class RadixOptimizedConversation:
    """
    Multi-turn conversation manager optimized for RadixAttention.
    
    Key optimizations:
    1. Consistent message formatting (same format = same prefix tokens)
    2. Session ID-based prefix grouping (related conversations share more)
    3. Minimal history modification (append-only when possible)
    """
    
    def __init__(self, system_prompt: str):
        self.system_prompt = system_prompt
        self.messages: List[Dict[str, str]] = []
        self.prefix_hash: Optional[str] = None
    
    def _format_message(self, role: str, content: str) -> str:
        """
        Format a message consistently.
        
        CRITICAL: Use exact same format every time!
        Different formatting = different tokens = no cache hit
        """
        # Use Llama-style formatting
        if role == "system":
            return f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{content}<|eot_id|>"
        elif role == "user":
            return f"<|start_header_id|>user<|end_header_id|>\n\n{content}<|eot_id|>"
        elif role == "assistant":
            return f"<|start_header_id|>assistant<|end_header_id|>\n\n{content}<|eot_id|>"
        else:
            raise ValueError(f"Unknown role: {role}")
    
    def build_prompt(self) -> str:
        """
        Build the full conversation prompt.
        
        Structure designed for maximum cache reuse:
        [System Prompt] - Cached across ALL conversations with same system
        [Turn 1] - Cached for this conversation
        [Turn 2] - Cached for this conversation
        [...]
        [Current Turn] - New content
        """
        parts = [self._format_message("system", self.system_prompt)]
        
        for msg in self.messages:
            parts.append(self._format_message(msg["role"], msg["content"]))
        
        # Add generation prompt
        parts.append("<|start_header_id|>assistant<|end_header_id|>\n\n")
        
        return "".join(parts)
    
    def add_user_message(self, content: str):
        """Add a user message to the conversation."""
        self.messages.append({"role": "user", "content": content})
    
    def add_assistant_message(self, content: str):
        """Add an assistant message to the conversation."""
        self.messages.append({"role": "assistant", "content": content})
    
    def get_prefix_hash(self) -> str:
        """
        Get a hash of the current prefix (system + history).
        
        Same hash = high likelihood of cache hit.
        """
        prefix = self._format_message("system", self.system_prompt)
        for msg in self.messages[:-1] if self.messages else []:
            prefix += self._format_message(msg["role"], msg["content"])
        
        return hashlib.md5(prefix.encode()).hexdigest()[:8]
    
    def estimate_cache_benefit(self) -> dict:
        """
        Estimate the caching benefit for this conversation.
        """
        full_prompt = self.build_prompt()
        system_len = len(self._format_message("system", self.system_prompt))
        
        # Calculate cacheable portion
        if len(self.messages) > 0:
            # All but the last user message is cacheable
            last_msg_len = len(self._format_message(
                self.messages[-1]["role"], 
                self.messages[-1]["content"]
            ))
            cacheable_len = len(full_prompt) - last_msg_len - 50  # -50 for gen prompt
        else:
            cacheable_len = system_len
        
        return {
            "total_chars": len(full_prompt),
            "cacheable_chars": cacheable_len,
            "cache_ratio": cacheable_len / len(full_prompt),
            "turn_number": len(self.messages) // 2 + 1
        }


class MultiConversationManager:
    """
    Manage multiple conversations with shared prefix optimization.
    
    Conversations with the same system prompt share the cached prefix.
    """
    
    def __init__(self):
        self.conversations: Dict[str, RadixOptimizedConversation] = {}
        self.system_prompts: Dict[str, str] = {}  # Cache system prompts
    
    def create_conversation(self, session_id: str, 
                           system_prompt_key: str = "default") -> RadixOptimizedConversation:
        """
        Create a new conversation.
        
        Using system_prompt_key ensures conversations with same key
        share the exact same system prompt = better caching.
        """
        if system_prompt_key not in self.system_prompts:
            self.system_prompts[system_prompt_key] = self._get_default_system_prompt()
        
        conv = RadixOptimizedConversation(self.system_prompts[system_prompt_key])
        self.conversations[session_id] = conv
        return conv
    
    def _get_default_system_prompt(self) -> str:
        return "You are a helpful AI assistant. Be concise and accurate."
    
    def analyze_cache_sharing(self) -> dict:
        """
        Analyze cache sharing across all active conversations.
        """
        prefix_groups = {}
        for session_id, conv in self.conversations.items():
            prefix_hash = conv.get_prefix_hash()
            if prefix_hash not in prefix_groups:
                prefix_groups[prefix_hash] = []
            prefix_groups[prefix_hash].append(session_id)
        
        return {
            "total_conversations": len(self.conversations),
            "unique_prefixes": len(prefix_groups),
            "sharing_ratio": len(self.conversations) / len(prefix_groups) if prefix_groups else 0,
            "prefix_groups": prefix_groups
        }


# Demonstrate multi-turn optimization
print("📝 Multi-Turn Conversation Optimization Demo")
print("=" * 50)

# Create a conversation
conv = RadixOptimizedConversation(
    "You are a Python programming tutor. Explain concepts clearly."
)

# Simulate a multi-turn conversation
turns = [
    ("user", "What is a list in Python?"),
    ("assistant", "A list is an ordered, mutable collection of items."),
    ("user", "How do I add items to it?"),
    ("assistant", "Use append() to add one item, or extend() for multiple."),
    ("user", "What about removing items?"),
]

for role, content in turns:
    if role == "user":
        conv.add_user_message(content)
    else:
        conv.add_assistant_message(content)
    
    benefit = conv.estimate_cache_benefit()
    print(f"\nTurn {benefit['turn_number']}:")
    print(f"  Prefix hash: {conv.get_prefix_hash()}")
    print(f"  Cache ratio: {benefit['cache_ratio']:.1%}")
    print(f"  Cacheable: {benefit['cacheable_chars']:,} / {benefit['total_chars']:,} chars")

print("\n💡 Key Insight: As conversation grows, cache ratio increases!")
print("   Each new turn reuses ALL previous turns from cache.")

---

## Exercise 4: Production Deployment Configuration

**Task**: Create a production-ready SGLang deployment configuration with proper scaling and monitoring.

In [ ]:
import json

# Production Docker Compose configuration
docker_compose_config = """
version: '3.8'

services:
  sglang:
    image: lmsysorg/sglang:latest
    runtime: nvidia
    deploy:
      resources:
        reservations:
          devices:
            - driver: nvidia
              count: 1
              capabilities: [gpu]
    environment:
      - CUDA_VISIBLE_DEVICES=0
    ports:
      - "30000:30000"  # Main API
      - "30001:30001"  # Metrics
    volumes:
      - ~/.cache/huggingface:/root/.cache/huggingface
      - ./logs:/app/logs
    command: >
      python -m sglang.launch_server
      --model Qwen/Qwen3-8B-Instruct
      --host 0.0.0.0
      --port 30000
      --dtype bfloat16
      --context-length 8192
      --mem-fraction-static 0.85
      --max-running-requests 64
      --schedule-policy fcfs
      --enable-metrics
      --log-level info
      --enforce-eager
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:30000/health"]
      interval: 30s
      timeout: 10s
      retries: 3
      start_period: 120s
    restart: unless-stopped
    logging:
      driver: "json-file"
      options:
        max-size: "100m"
        max-file: "5"

  # Nginx load balancer (for multi-instance)
  nginx:
    image: nginx:alpine
    ports:
      - "8080:80"
    volumes:
      - ./nginx.conf:/etc/nginx/nginx.conf:ro
    depends_on:
      - sglang
    restart: unless-stopped

  # Prometheus for metrics
  prometheus:
    image: prom/prometheus:latest
    ports:
      - "9090:9090"
    volumes:
      - ./prometheus.yml:/etc/prometheus/prometheus.yml:ro
    restart: unless-stopped
"""

# Nginx configuration for load balancing
nginx_config = """
events {
    worker_connections 1024;
}

http {
    upstream sglang_backend {
        # Least connections for better load distribution
        least_conn;
        
        server sglang:30000;
        # Add more servers for multi-GPU:
        # server sglang-2:30000;
        # server sglang-3:30000;
        
        keepalive 32;
    }

    server {
        listen 80;
        
        # Health check endpoint
        location /health {
            proxy_pass http://sglang_backend/health;
            proxy_http_version 1.1;
        }
        
        # API endpoints
        location /v1/ {
            proxy_pass http://sglang_backend/v1/;
            proxy_http_version 1.1;
            proxy_set_header Connection "";
            proxy_set_header Host $host;
            
            # SSE streaming support
            proxy_buffering off;
            proxy_cache off;
            chunked_transfer_encoding on;
            
            # Timeouts for long generations
            proxy_read_timeout 300s;
            proxy_send_timeout 300s;
        }
        
        # Rate limiting
        limit_req_zone $binary_remote_addr zone=api:10m rate=10r/s;
        
        location /v1/completions {
            limit_req zone=api burst=20 nodelay;
            proxy_pass http://sglang_backend/v1/completions;
            proxy_buffering off;
        }
    }
}
"""

# Prometheus configuration
prometheus_config = """
global:
  scrape_interval: 15s
  evaluation_interval: 15s

scrape_configs:
  - job_name: 'sglang'
    static_configs:
      - targets: ['sglang:30001']
    metrics_path: /metrics
"""

# Health check and monitoring script
health_check_script = '''
#!/usr/bin/env python3
"""Production health check and monitoring for SGLang."""

import requests
import time
import sys

def check_health(url: str = "http://localhost:30000") -> dict:
    """Check SGLang server health."""
    try:
        # Basic health
        health_resp = requests.get(f"{url}/health", timeout=5)
        health_ok = health_resp.status_code == 200
        
        # Test inference
        start = time.time()
        test_resp = requests.post(
            f"{url}/v1/completions",
            json={
                "model": "default",
                "prompt": "Hello",
                "max_tokens": 1
            },
            timeout=30
        )
        latency_ms = (time.time() - start) * 1000
        inference_ok = test_resp.status_code == 200
        
        return {
            "healthy": health_ok and inference_ok,
            "health_endpoint": health_ok,
            "inference_working": inference_ok,
            "test_latency_ms": latency_ms
        }
    except Exception as e:
        return {
            "healthy": False,
            "error": str(e)
        }

if __name__ == "__main__":
    result = check_health()
    print(f"Health Check: {\"✓\" if result[\"healthy\"] else \"✗\"}")
    for k, v in result.items():
        print(f"  {k}: {v}")
    sys.exit(0 if result["healthy"] else 1)
'''

print("📦 Production Deployment Configuration")
print("=" * 50)
print("\nFiles to create:")
print("  1. docker-compose.yml - Main deployment config")
print("  2. nginx.conf - Load balancer config")
print("  3. prometheus.yml - Metrics collection")
print("  4. health_check.py - Monitoring script")

print("\n🔑 Key Production Settings:")
print("  • --mem-fraction-static 0.85: Reserve 85% GPU memory")
print("  • --max-running-requests 64: Limit concurrent requests")
print("  • --enforce-eager: Required for DGX Spark ARM64")
print("  • Health checks every 30s with 120s startup grace period")
print("  • Log rotation: 100MB max, 5 files")

print("\n🚀 Deployment Commands:")
print("  # Start the stack")
print("  docker-compose up -d")
print("")
print("  # Check logs")
print("  docker-compose logs -f sglang")
print("")
print("  # Scale (multi-GPU)")
print("  docker-compose up -d --scale sglang=2")

---

## Key Takeaways

1. **Prefix Design is Critical**: Put static content first, dynamic content last

2. **Consistent Formatting**: Same format = same tokens = cache hits

3. **Multi-Turn Benefits**: Each conversation turn builds on cached history

4. **Production Considerations**:
   - Use `--enforce-eager` on DGX Spark
   - Set appropriate memory fractions
   - Implement health checks and monitoring
   - Use load balancing for multiple instances

5. **Monitoring**: Track cache hit rates to validate optimization effectiveness